***GENERATED CODE FOR tsdata PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
import pyspark
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = transformationDF.drop('_c0')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from prophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run tsdataHooks.ipynb
try:
	#sourcePreExecutionHook()

	datetimedfmxmj = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1718183408943_Datetime_df.csv', 'filename': 'Datetime_df.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/TESTING/Datetime_df.csv', 'viewFileName': 'Datetime_df.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run tsdataHooks.ipynb
try:
	#transformationPreExecutionHook()

	tsdatafeatureforecast = TransformationTimeSeriesForecastingMain.run(datetimedfmxmj, json.dumps( {"FE": {"functionList": [{"function": "Original"}], "stage_attributes": {}, "featureList": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "_c0", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "99", "mean": "49.0", "stddev": "28.72", "min": "0", "max": "98", "missing": "0"}, "updatedLabel": "_c0"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lengthofstay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "4.1", "stddev": "2.48", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "lengthofstay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rcount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "1.37", "stddev": "1.71", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "rcount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hematocrit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "11.53", "stddev": "2.02", "min": "7.0", "max": "21.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hematocrit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "neutrophils", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "9.46", "stddev": "3.99", "min": "4.0", "max": "32.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "neutrophils"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sodium", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "136.78", "stddev": "2.99", "min": "131.0", "max": "145.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "glucose", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "141.51", "stddev": "29.58", "min": "77.0", "max": "248.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "glucose"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bloodureanitro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "14.34", "stddev": "9.61", "min": "3.0", "max": "83.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bloodureanitro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "0.69", "stddev": "0.47", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "29.05", "stddev": "1.86", "min": "24.0", "max": "33.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pulse", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "72.8", "stddev": "10.1", "min": "51", "max": "93", "missing": "0"}, "updatedLabel": "pulse"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "respiration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "6.09", "stddev": "0.59", "min": "4.0", "max": "8.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "respiration"}, {"transformationsData": [{}], "feature": "AdmitDate_converttodate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AdmitDate_converttodate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "gender_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "0.44", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "gender_stringindexer"}], "features": {"timecolumn": "AdmitDate_converttodate", "tocompare": "lengthofstay"}, "dataPercentage": "100", "originalfile": "/FileStore/platform/testdata/1718183408943_Datetime_df.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(tsdatafeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run tsdataHooks.ipynb
try:
	#mlPreExecutionHook()

	model = driverProphet(tsdatafeatureforecast)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

spark.stop()


